#**이번 실습 과제3에서는 HuggingFace 라이브러리를 활용하여,생성형 인공지능(Generative AI)의 두 가지 핵심적인 응용 분야인 컴퓨터 비전(Computer Vision)과 자연어 처리(Natural Language Processing)를 실습해보겠습니다.**#

## **메뉴 → 런타임 → 런타임 유형 변경 → GPU로 설정해야 에러없이 진행됩니다**##

###**Problem 1. Machine Translation & Image Generation**

이 실습 과제에서는 자연어 처리 기술을 사용하여 텍스트를 번역하는 기계번역(Machine Translation) 작업을 수행 후 이를 기반으로 실제 이미지 생성(Image Generation)이 어떻게 작동되는지를 실습합니다.

창의적인 텍스트 프롬프트(한글)를 제공하여, 이를 기반으로 실제 이미지를 생성해보세요.

In [ ]:
!pip install --upgrade -qq diffusers transformers[sentencepiece] sacremoses accelerate scipy safetensors gensim

In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-ko-en", clean_up_tokenization_spaces=True)
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-ko-en")

In [9]:
prompt = "" # Q1. 번역하고자 하는 문장을 한글로 작성하세요.
tokenized_text = tokenizer(prompt, return_tensors='pt')

translation_tokens = model.generate(**tokenized_text)
translated_text = tokenizer.decode(translation_tokens[0], skip_special_tokens=True)

In [ ]:
import warnings
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler

warnings.filterwarnings("ignore", category=FutureWarning)
pipe = StableDiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-2-1", torch_dtype=torch.float16)
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe = pipe.to("cuda")

In [ ]:
image = pipe(translated_text).images[0]

image.save(f"2024_AI_BU.png")

print(f"Orginal Prompt: {prompt}\nTranslated Prompt: {translated_text}\n")
image

###**Problem 2. Sentiment Analysis**
이 실습 과제에서는 자연어 처리 기술을 사용하여 문장의 감성 분석(Sentiment Analysis) 작업을 수행합니다.

영어로 된 문장 3개를 제공하여, 각 해당 문장의 감성 분석을 실습합니다.

In [ ]:
import warnings
from transformers import pipeline

model_name = "cardiffnlp/twitter-roberta-base-sentiment"

warnings.filterwarnings("ignore", category=FutureWarning)
sentiment_analysis = pipeline("sentiment-analysis", model=model_name, device=0)

In [16]:
def convert_to_sentiment_label(label):
    if label == "LABEL_2":
        label = "긍정"
    elif label == "LABEL_1":
        label = "중립"
    elif label == "LABEL_0":
        label = "부정"
    return label

sentences = [
    "",
    "",
    ""
] # Q2. 분석하고자 하는 문장 3개를 각각 영어로 작성하세요.


In [ ]:
results = sentiment_analysis(sentences)

for sentence, result in zip(sentences, results):
  label = result['label']
  score = result['score']
  converted_label = convert_to_sentiment_label(label)
  print(f"Text: {sentence}\nSentiment: {converted_label}\nProbability: {round(score * 100, 2)}%\n")